Se tomara como base https://www.kaggle.com/joanfo/word2vec-cnn-extra-features-0-9757

In [1]:
import pandas as pd
from   keras                        import initializers, regularizers, constraints, callbacks, optimizers
from   keras.layers                 import Conv1D, Embedding, GlobalMaxPooling1D, concatenate, Input, Dense
from   keras.models                 import Sequential, Model
from   keras.preprocessing.sequence import pad_sequences
from   keras.preprocessing.text     import Tokenizer
from   sklearn.model_selection      import train_test_split
from   gensim.models.keyedvectors   import KeyedVectors
import numpy as np

Traigo Los Datos

In [2]:
from google.colab import files
import io

In [18]:
uploaded = files.upload()

Saving test_pre_processing_3.csv to test_pre_processing_3 (1).csv
Saving train_pre_processing_3.csv to train_pre_processing_3 (1).csv
Saving twiter_with_data_location.csv to twiter_with_data_location.csv


In [21]:
train_pre_processing_3= pd.read_csv(io.StringIO(uploaded['train_pre_processing_3.csv'].decode('utf-8')))

In [20]:
tw_test = pd.read_csv(io.StringIO(uploaded['test_pre_processing_3.csv'].decode('utf-8')))
twiter_my2=pd.read_csv(io.StringIO(uploaded['twiter_with_data_location.csv'].decode('utf-8')))

In [22]:
twiter_my2["text_contain_keyword"]=train_pre_processing_3.text_contain_keyword

In [23]:
twiter_my2["len_text"]=twiter_my2.text.apply(len)

In [24]:
twiter_my2.len_text.head(2)

0    69
1    38
Name: len_text, dtype: int64

In [25]:
def countWords(words):
  words=re.split(r'\W+', words)
  cont=0
  unique_words=[]
  for word in words:
    if (word not in unique_words):
      cont=cont+1
      unique_words.append(word)
  return cont

In [26]:
import re

In [27]:
twiter_my2["count_word_uniques"]=twiter_my2.text.apply(lambda x: countWords(x))

In [28]:
twiter_my2.count_word_uniques

0       13
1        7
2       18
3       10
4       16
        ..
7608    14
7609    18
7610    14
7611    22
7612    15
Name: count_word_uniques, Length: 7613, dtype: int64

In [29]:
tw_test["len_text"]=tw_test.text.apply(len)
tw_test["count_word_uniques"]=tw_test.text.apply(lambda x: countWords(x))

In [30]:
twiter_my2.head(2)

,Unnamed: 0,id,keyword,location,text,target,location_isnull,location_free,address_latitude_longitude,latitude,longitude,city,country,text_contain_keyword,len_text,count_word_uniques
0,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,True,NaN,NaN,NaN,NaN,NaN,NaN,False,69,13
1,1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,True,NaN,NaN,NaN,NaN,NaN,NaN,False,38,7


In [31]:
train=twiter_my2.loc[:,["id","text","len_text","count_word_uniques","text_contain_keyword"]]  
test=   tw_test.loc[:, ["id","text", "len_text", "count_word_uniques","text_contain_keyword"]]  

In [32]:
total_words = train.loc[:, 'id' : 'text']
total_words=pd.concat([total_words,test], sort = 'False')


In [33]:
total_words.head(2)

,count_word_uniques,id,len_text,text,text_contain_keyword
0,NaN,1,NaN,Our Deeds are the Reason of this #earthquake M...,NaN
1,NaN,4,NaN,Forest fire near La Ronge Sask. Canada,NaN


**Comenzando a Tokenizar**

In [34]:
numWords=20000
tokenizer = Tokenizer(num_words = numWords)

In [35]:
tokenizer.fit_on_texts(total_words['text'])

In [36]:
sequences_train  = tokenizer.texts_to_sequences(train['text'])
sequences_test   = tokenizer.texts_to_sequences(test['text'])

In [37]:
word_index       = tokenizer.word_index

In [38]:
#max_length       = 200
max_length       = 300
x_train          = pad_sequences(sequences_train, maxlen = max_length)
x_test           = pad_sequences(sequences_test, maxlen = max_length)
x_features_train = train[['len_text', 'count_word_uniques']]
x_features_test  = test[['len_text', 'count_word_uniques']]
y_train          = twiter_my2["target"].values

Cargo Word2Vect

In [39]:
#word_vectors se obtiene de word2vec, que asigna casi todas las palabras en inglés a un tamaño de 300
#vectores que tiene una corta distancia cos con palabras cercanas cercanas
word_vectors = KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary = True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


**Comienzo la creacion de mi matriz embedign**

In [40]:
#pongo cada palabra en word_index para tener una matriz donde cada fila i contendra el correspondiente vector obtenido del Word2vect
#embeddind_dimension = 300
embeddind_dimension = 300
vocabulary_size = min(len(word_index) + 1, numWords)
embedding_matrix = np.zeros((vocabulary_size, embeddind_dimension))

In [41]:
for word, i in word_index.items():
    if i < numWords:
        try:
            embedding_vector = word_vectors[word]
            embedding_matrix[i] = embedding_vector
        except KeyError:
            embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), embeddind_dimension)

del(word_vectors)

**Comienzo a crear mi modelo**

In [42]:
input_embedding = Input(shape = (max_length,))

In [43]:
embedding = Embedding(vocabulary_size,
                      embeddind_dimension,
                      weights = [embedding_matrix],
                      trainable = False)(input_embedding)

**Usando un modelo Secuencial**

In [44]:
e = Embedding(len(embedding_matrix), 200, input_length=300)

In [45]:
from keras.layers import Dropout

In [46]:
model1 = Sequential()
model1.add(e)
model1.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model1.add(GlobalMaxPooling1D())
model1.add(Dropout(0.25))
model1.add(Dense(256, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 200)          4000000   
_________________________________________________________________
conv1d (Conv1D)              (None, 299, 100)          40100     
_________________________________________________________________
global_max_pooling1d (Global (None, 100)               0         
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               25856     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 4,066,213
Trainable params: 4,066,213
Non-trainable params: 0
______________________________________________

In [47]:
Xtrain1, Xval1, ytrain1, yval1 = train_test_split(x_train, y_train,
                                              train_size = 0.95, random_state = 233)

Xfeaturestrain1, Xfeaturesval1, ytrain1, yval1 = train_test_split(x_features_train, y_train,
                                                            train_size = 0.95, random_state = 233)

In [48]:
model1.fit(Xtrain1, ytrain1,
          validation_data=(Xval1, yval1), 
          epochs=5, 
          batch_size=300,
          verbose=1)

Epoch 1/5
25/25 [==============================] - 15s 615ms/step - loss: 0.6578 - accuracy: 0.6157 - val_loss: 0.5860 - val_accuracy: 0.7113
Epoch 2/5
25/25 [==============================] - 15s 608ms/step - loss: 0.4965 - accuracy: 0.7659 - val_loss: 0.4504 - val_accuracy: 0.7979
Epoch 3/5
25/25 [==============================] - 15s 607ms/step - loss: 0.2765 - accuracy: 0.8963 - val_loss: 0.4305 - val_accuracy: 0.8136
Epoch 4/5
25/25 [==============================] - 15s 606ms/step - loss: 0.1330 - accuracy: 0.9577 - val_loss: 0.4826 - val_accuracy: 0.8136
Epoch 5/5
25/25 [==============================] - 15s 606ms/step - loss: 0.0621 - accuracy: 0.9813 - val_loss: 0.5566 - val_accuracy: 0.7900


In [49]:
y_pred1 = model1.predict_classes([x_test, x_features_test])

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [50]:
y_pred1

array([[1],
       [0],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [51]:
submit1 = tw_test['id'].to_frame()
submit1['target'] = pd.DataFrame(y_pred1)

In [52]:
submit1.to_csv('submit1.csv', index=False)
files.download('submit1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Usando un modelo Model**


In [53]:
conv1 = Conv1D(filters = 100, kernel_size = 2, padding = 'same')(embedding)
conv2 = Conv1D(filters = 100, kernel_size = 3, padding = 'same')(embedding)
conv3 = Conv1D(filters = 100, kernel_size = 4, padding = 'same')(embedding)

pool1 = GlobalMaxPooling1D()(conv1)
pool2 = GlobalMaxPooling1D()(conv2)
pool2 = GlobalMaxPooling1D()(conv3)

In [54]:
input_features = Input(shape = (2,))
merged_tensor = concatenate([pool1, pool2, pool2, input_features], axis = 1)
output = Dense(units = 1, activation = 'sigmoid')(merged_tensor)
model2 = Model(inputs = [input_embedding, input_features], outputs = output)

In [55]:
model2.compile(loss      = 'binary_crossentropy',
              optimizer = optimizers.Adam(),
              metrics   = ['accuracy'])

In [56]:
Xtrain, Xval, ytrain, yval = train_test_split(x_train, y_train,
                                              train_size = 0.95, random_state = 233)

Xfeaturestrain, Xfeaturesval, ytrain, yval = train_test_split(x_features_train, y_train,
                                                            train_size = 0.95, random_state = 233)

In [57]:
model2.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 300, 300)     6000000     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 300, 100)     60100       embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 300, 100)     120100      embedding[0][0]                  
_______________________________________________________________________________________

In [58]:
model2.fit(x = [Xtrain, Xfeaturestrain], y = ytrain, epochs=5,
          validation_data = ([Xval, Xfeaturesval], yval), verbose = 1)

Epoch 1/5
226/226 [==============================] - 44s 194ms/step - loss: 1.2686 - accuracy: 0.6025 - val_loss: 0.6331 - val_accuracy: 0.6982
Epoch 2/5
226/226 [==============================] - 44s 193ms/step - loss: 0.4443 - accuracy: 0.8024 - val_loss: 0.4972 - val_accuracy: 0.7664
Epoch 3/5
226/226 [==============================] - 44s 193ms/step - loss: 0.3073 - accuracy: 0.8803 - val_loss: 0.5222 - val_accuracy: 0.7480
Epoch 4/5
226/226 [==============================] - 44s 193ms/step - loss: 0.2155 - accuracy: 0.9299 - val_loss: 0.5405 - val_accuracy: 0.7585
Epoch 5/5
226/226 [==============================] - 44s 193ms/step - loss: 0.1555 - accuracy: 0.9542 - val_loss: 0.5652 - val_accuracy: 0.7454


In [59]:
y_pred2 = model2.predict([x_test, x_features_test])

In [61]:
#y_pred2

In [46]:
#import math

In [62]:
submit2 = tw_test['id'].to_frame()
submit2['target'] = pd.DataFrame(y_pred2)

In [63]:
#submit.target=submit.target.apply(lambda x: math.ceil(x))
submit2.target=submit2.target.apply(lambda x: round(x))
submit2

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [64]:
submit2.to_csv('submit2.csv', index=False)
files.download('submit2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [65]:
model3 = Sequential()
model3.add(e)
model3.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model3.add(GlobalMaxPooling1D())
model3.add(Dense(200, activation='relu'))
model3.add(Dropout(0.25))
model3.add(Dense(200, activation='relu'))
model3.add(Dense(100, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 200)          4000000   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 299, 100)          40100     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_5 (Dense)              (None, 100)              

In [66]:
model3.fit(x = [Xtrain, Xfeaturestrain], y = ytrain, epochs=5,
          validation_data = ([Xval, Xfeaturesval], yval), verbose = 1)

Epoch 1/5
226/226 [==============================] - 25s 111ms/step - loss: 0.2176 - accuracy: 0.9125 - val_loss: 0.5495 - val_accuracy: 0.7507
Epoch 2/5
226/226 [==============================] - 28s 122ms/step - loss: 0.0639 - accuracy: 0.9805 - val_loss: 0.6604 - val_accuracy: 0.7822
Epoch 3/5
226/226 [==============================] - 25s 109ms/step - loss: 0.0330 - accuracy: 0.9891 - val_loss: 0.7913 - val_accuracy: 0.7927
Epoch 4/5
226/226 [==============================] - 25s 110ms/step - loss: 0.0239 - accuracy: 0.9913 - val_loss: 0.8716 - val_accuracy: 0.7533
Epoch 5/5
226/226 [==============================] - 25s 110ms/step - loss: 0.0151 - accuracy: 0.9923 - val_loss: 0.9120 - val_accuracy: 0.7585


In [67]:

y_pred3 = model3.predict_classes([x_test, x_features_test])

In [68]:
y_pred3

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [69]:
submit3 = tw_test['id'].to_frame()
submit3['target'] = pd.DataFrame(y_pred3)

In [70]:
submit3.to_csv('submit3.csv', index=False)
files.download('submit3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [71]:
model4 = Sequential()
model4.add(e)
model4.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model4.add(GlobalMaxPooling1D())
model4.add(Dense(200, activation='relu'))
model4.add(Dropout(0.25))
model4.add(Dense(200, activation='relu'))
model4.add(Dense(100, activation='relu'))
model4.add(Dense(50, activation='relu'))
model4.add(Dropout(0.25))
model4.add(Dense(1, activation='sigmoid'))
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 200)          4000000   
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 299, 100)          40100     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 100)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 200)               20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_9 (Dense)              (None, 100)              

In [72]:
model4.fit(x = [Xtrain, Xfeaturestrain], y = ytrain, epochs=5,
          validation_data = ([Xval, Xfeaturesval], yval), verbose = 1)

Epoch 1/5
226/226 [==============================] - 25s 111ms/step - loss: 0.1253 - accuracy: 0.9499 - val_loss: 0.7397 - val_accuracy: 0.7454
Epoch 2/5
226/226 [==============================] - 25s 111ms/step - loss: 0.0325 - accuracy: 0.9899 - val_loss: 1.0122 - val_accuracy: 0.7507
Epoch 3/5
226/226 [==============================] - 25s 110ms/step - loss: 0.0208 - accuracy: 0.9921 - val_loss: 1.1358 - val_accuracy: 0.7454
Epoch 4/5
226/226 [==============================] - 25s 110ms/step - loss: 0.0145 - accuracy: 0.9924 - val_loss: 1.3605 - val_accuracy: 0.7454
Epoch 5/5
226/226 [==============================] - 25s 110ms/step - loss: 0.0113 - accuracy: 0.9927 - val_loss: 1.7976 - val_accuracy: 0.7559


In [73]:
y_pred4 = model4.predict_classes([x_test, x_features_test])

In [74]:
y_pred4

array([[0],
       [0],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [75]:
submit4 = tw_test['id'].to_frame()
submit4['target'] = pd.DataFrame(y_pred4)

In [76]:
submit4

,id,target
0,0,0
1,2,0
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [77]:
submit4.to_csv('submit4.csv', index=False)
files.download('submit4.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [78]:
model5 = Sequential()
model5.add(e)
model5.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model5.add(GlobalMaxPooling1D())
model5.add(Dense(200, activation='relu'))
model5.add(Dropout(0.25))
model5.add(Dense(200, activation='relu'))
model5.add(Dense(50, activation='relu'))
model5.add(Dropout(0.5))
model5.add(Dense(1, activation='sigmoid'))
model5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model5.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 200)          4000000   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 299, 100)          40100     
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 100)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 200)               20200     
_________________________________________________________________
dropout_4 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_14 (Dense)             (None, 50)               

In [79]:
model5.fit(x = [Xtrain, Xfeaturestrain], y = ytrain, epochs=5,
          validation_data = ([Xval, Xfeaturesval], yval), verbose = 1)

Epoch 1/5
226/226 [==============================] - 25s 110ms/step - loss: 0.0980 - accuracy: 0.9649 - val_loss: 1.3318 - val_accuracy: 0.7402
Epoch 2/5
226/226 [==============================] - 25s 109ms/step - loss: 0.0249 - accuracy: 0.9916 - val_loss: 1.2324 - val_accuracy: 0.7585
Epoch 3/5
226/226 [==============================] - 25s 109ms/step - loss: 0.0166 - accuracy: 0.9917 - val_loss: 1.9494 - val_accuracy: 0.7507
Epoch 4/5
226/226 [==============================] - 25s 109ms/step - loss: 0.0101 - accuracy: 0.9929 - val_loss: 2.4223 - val_accuracy: 0.7480
Epoch 5/5
226/226 [==============================] - 25s 109ms/step - loss: 0.0106 - accuracy: 0.9929 - val_loss: 2.6067 - val_accuracy: 0.7428


In [80]:
y_pred5 = model5.predict_classes([x_test, x_features_test])

In [81]:
submit5 = tw_test['id'].to_frame()
submit5['target'] = pd.DataFrame(y_pred5)

In [82]:
submit5.to_csv('submit5.csv', index=False)
files.download('submit5.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [83]:
#uploaded2 = files.upload()

Saving submit_kaggle_2.csv to submit_kaggle_2.csv
Saving submit_kaggle_4.csv to submit_kaggle_4.csv
Saving submit_kaggle_5.csv to submit_kaggle_5.csv
Saving submit_kaggle_Combinate_6.csv to submit_kaggle_Combinate_6.csv
Saving submitf.csv to submitf.csv
Saving Sumbit_Kaggle_1.csv to Sumbit_Kaggle_1.csv


In [86]:
#submit_kaggle_false = pd.read_csv(io.StringIO(uploaded2['submit_kaggle_false.csv'].decode('utf-8')))
#submit_kaggle_true = pd.read_csv(io.StringIO(uploaded2['submit_kaggle_true.csv'].decode('utf-8')))
#submit_kaggle_true_false = pd.read_csv(io.StringIO(uploaded2['submit_kaggle_true_false.csv'].decode('utf-8')))
#submit_kaggle_combinate = pd.read_csv(io.StringIO(uploaded2['submit_kaggle_combinate.csv'].decode('utf-8')))
#submitf = pd.read_csv(io.StringIO(uploaded2['submitf.csv'].decode('utf-8')))

In [87]:
#uploaded3 = files.upload()

In [93]:
#submit_kaggle_combinate1 = pd.read_csv(io.StringIO(uploaded3['submit_kaggle.csv'].decode('utf-8')))
#submit_kaggle_combinate2 = pd.read_csv(io.StringIO(uploaded3['submit_kaggle_combinate.csv'].decode('utf-8')))
#submit_kaggle_combinate3 = pd.read_csv(io.StringIO(uploaded3['submit_kaggle2.csv'].decode('utf-8')))

USANDO RESULTADOS PRINCIPALES

In [88]:
uploaded4 = files.upload()

Saving submit_kaggle_2.csv to submit_kaggle_2 (1).csv
Saving submit_kaggle_4.csv to submit_kaggle_4 (1).csv
Saving submit_kaggle_5.csv to submit_kaggle_5 (1).csv
Saving submit_kaggle_Combinate_6.csv to submit_kaggle_Combinate_6 (1).csv
Saving submitf.csv to submitf (1).csv
Saving Sumbit_Kaggle_1.csv to Sumbit_Kaggle_1 (1).csv


In [93]:
uploaded5 = files.upload()

Saving submit1.csv to submit1 (2).csv
Saving submit2.csv to submit2 (2).csv
Saving submit3.csv to submit3 (1).csv
Saving submit4.csv to submit4 (1).csv
Saving submit5.csv to submit5 (2).csv
Saving submitf.csv to submitf (2).csv


In [94]:
submit_kaggle_2 = pd.read_csv(io.StringIO(uploaded4['submit_kaggle_2.csv'].decode('utf-8')))
submit_kaggle_4 = pd.read_csv(io.StringIO(uploaded4['submit_kaggle_4.csv'].decode('utf-8')))
submit_kaggle_5 = pd.read_csv(io.StringIO(uploaded4['submit_kaggle_5.csv'].decode('utf-8')))
submit_kaggle_Combinate_6 = pd.read_csv(io.StringIO(uploaded4['submit_kaggle_Combinate_6.csv'].decode('utf-8')))
Sumbit_Kaggle_1 = pd.read_csv(io.StringIO(uploaded4['Sumbit_Kaggle_1.csv'].decode('utf-8')))
submitf = pd.read_csv(io.StringIO(uploaded5['submitf.csv'].decode('utf-8')))
submit1 = pd.read_csv(io.StringIO(uploaded5['submit1.csv'].decode('utf-8')))
submit2 = pd.read_csv(io.StringIO(uploaded5['submit2.csv'].decode('utf-8')))
submit3 = pd.read_csv(io.StringIO(uploaded5['submit3.csv'].decode('utf-8')))
submit4= pd.read_csv(io.StringIO(uploaded5['submit4.csv'].decode('utf-8')))
submit5 = pd.read_csv(io.StringIO(uploaded5['submit5.csv'].decode('utf-8')))

In [95]:
submitFinal = tw_test['id'].to_frame()

In [100]:
submitFinal["pred1"]=submit_kaggle_2.target
submitFinal["pred2"]=submit_kaggle_4.target
submitFinal["pred3"]=submit_kaggle_5.target
submitFinal["pred4"]=submit_kaggle_Combinate_6.target
submitFinal["pred5"]=Sumbit_Kaggle_1.target
submitFinal["pred6"]=submit1.target
submitFinal["pred7"]=submit2.target
submitFinal["pred8"]=submit3.target
submitFinal["pred9"]=submit4.target
submitFinal["pred10"]=submit5.target
submitFinal["pred11"]=submitf.target

In [101]:
submitFinal

,id,pred1,pred2,pred3,pred4,pred5,pred6,pred7,pred8,pred9,pred10,suma,target,pred11
0,0,1,1,1,1,0,1,1,1,0,1,9,1,1
1,2,1,1,1,1,0,0,1,1,0,0,8,1,1
2,3,1,1,1,1,1,1,1,1,1,0,11,1,1
3,9,1,1,1,1,0,1,0,1,1,1,9,1,1
4,11,1,1,1,1,1,1,1,1,1,1,11,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3258,10861,1,1,1,1,1,1,1,0,1,1,10,1,1
3259,10865,1,1,1,1,1,1,1,1,1,1,11,1,1
3260,10868,1,1,1,1,1,1,1,1,1,1,11,1,1
3261,10874,1,1,1,1,1,1,1,1,1,1,11,1,1


In [102]:
submitFinal["suma"]=submitFinal.pred1+submitFinal.pred2+submitFinal.pred3+submitFinal.pred3\
  +submitFinal.pred4+submitFinal.pred5+submitFinal.pred6+submitFinal.pred7+submitFinal.pred8+submitFinal.pred9\
  +submitFinal.pred10

In [103]:
submitFinal["target"]=submitFinal.suma.apply(lambda x: 1 if x>5 else 0)

In [104]:
submitf = submitFinal[['id',"target"]]
submitf.to_csv('submitf.csv', index=False)
files.download('submitf.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>